# Datacraft Events Assistant – Querying a Local Database with Natural Language

Welcome to this hands-on notebook!  
The goal is to build an intelligent assistant that can answer questions like (using a database of events extracted from Datacraft’s internal calendar):

> *"What are the upcoming events in French?"*  
> *"How many workshops are related to AI?"*

All of that by querying a **local database** using a **natural language interface** powered by an LLM (Large Language Model).


## What You'll Learn

In this notebook, you will:

Connect a natural language agent to a local SQLite database  
Define a tool that lets the agent run SQL queries  
Ask real questions and get structured answers

You’ll also learn how to use the [`smolagents`](https://github.com/smol-ai/smolagent) framework to combine tools with LLMs like GPT-4.


## The `events` Database

The agent will query a local database of events extracted from Datacraft’s internal calendar.  
The database is a simple SQLite file called `events.db`, and it contains a single table: **`evenements`**.

Each row represents one event (a workshop, talk, or training session).

### Main columns:

| Column         | Description                                        |
|----------------|----------------------------------------------------|
| `nomenclature` | Unique code for the event (e.g. `0706-AI-Intro`)   |
| `titre`        | Event title (e.g. `"Intro to Generative AI"`)      |
| `description`  | Full description of the event                      |
| `date`         | Date of the event (as text, e.g. `2024-06-03`)     |
| `horaire`      | Time range (e.g. `"9h00 - 17h30"`)                 |
| `tag`          | Topics or tags (e.g. `#AI`, `#DataGouv`)           |
| `langue`       | Language of the event (e.g. `"français"`, `"english"`) |


## Technologies Used

This notebook uses:

- `smolagent` – to manage the agent and tools
- `OpenAI` – to interpret your questions
- `SQLAlchemy` – to connect to the database


## Expected Outcome

By the end of this notebook, you’ll be able to write:

```python
agent.run("List all French events after July 1st.")

In [ ]:
import os
from smolagents import CodeAgent, OpenAIServerModel
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect

load_dotenv(override=True)

## Getting more info about the database column names

### TODO: 

Instanciate an inspector object and print the table column names.

In [ ]:
engine = create_engine("sqlite:///../../data/events.db")
inspector = ...
for col_info in inspector.get_columns("evenements"):
    ...

### Defining a Tool for the Agent: `query_events_db`

In `smolagent`, a **tool** is a Python function that the agent can call to interact with the outside world — such as querying a database, calling an API, reading a file, or performing calculations.

#### Why define a tool?

LLMs like GPT-4 don't have direct access to your local environment or files.  
By wrapping a function with the `@tool` decorator, you allow the agent to **"see" and use** that function when it decides it's relevant to the user’s question.


#### What does a tool need?

A tool in `smolagent` is simply a Python function, but it must follow these **three key rules**:

1. **A clear and unique name**  
   The name of the function is how the agent will refer to it internally. It should describe the action it performs.

2. **A well-written docstring**  
   The agent reads the docstring to understand:
   - what the tool does
   - how to use it
   - what arguments it accepts
   - what it returns  
   
   Without a good docstring, the agent will either ignore the tool or misuse it.

3. **A simple and reliable interface**  
   The tool should expect only standard Python types as arguments (`str`, `int`, `float`, etc.) and return a string or JSON-serializable output.

#### Our example: `query_events_db`

This tool allows the agent to send **raw SQL queries** to the `evenements` table in our local database.

You can ask the agent:
- `"What are the events in French?"`
- `"List all events that mention AI."`
- `"How many events are scheduled after July?"`

The agent will translate your question into SQL, run the query using this tool, and return the results in plain text.

### TODO: 

Go to [tools.py](./tools.py) and fill the tool called `query_events_db` that takes a SQL query as input and returns the result of the query.

Do not forget to fill the docstrings information with the column names you found above ^

In [ ]:
from workshop_smolagents_mcp.tools import query_events_db

We now set up the model that will power the agent.
We’re using OpenAI’s API to access the model gpt-4o.

In [ ]:
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)


Now that we have:

- a model (`gpt-4o` via OpenAI),
- a tool (`query_events_db`) that knows how to query the event database,  

we can combine them into a single `CodeAgent`.

The `CodeAgent` is the core component of the `smolagent` framework.  
It takes a list of tools and a language model, and automatically decides:

- When to call a tool
- How to use it
- What answer to return to the user


### TODO: 

Fill the `tools` list with the tool defined above to include the `query_events_db` tool in the agent.

In [ ]:
agent = CodeAgent(
    tools=[...],
    model=model,
)

let's try it ! 

### TODO: 

Have fun: try to run queries!

In [ ]:
question = "..."
agent.run(question)